In [2]:
import pandas as pd
import json

In [3]:
from ipywidgets import IntProgress
from IPython.display import display

### load data

In [9]:
{'id': 919945, 'type': 'message', 'date': '2019-11-13T13:20:26', 'from': 'Vladimir Korotkov', 'from_id': 4420223003, 'text': 'взять этих..'}

{'id': 919945,
 'type': 'message',
 'date': '2019-11-13T13:20:26',
 'from': 'Vladimir Korotkov',
 'from_id': 4420223003,
 'text': 'взять этих..'}

In [16]:
def load_from_telegram(telegram_file_name, group):
    
    with open(telegram_file_name) as f:
        file_data = json.load(f)

    prgBar = IntProgress(min = 0, max = len(file_data))
    display(prgBar)
        
    df = pd.DataFrame(columns=['from','from_id','text'])
    
    for msg in file_data['messages']:    
        if type(msg['text'])==type('') and len(msg['text'])>20 and len(msg['text'])<200:
            #if user == '' or msg['from']==user:
            df = df.append({'from':msg['from'],'from_id':msg['from_id'],'text':msg['text']}, ignore_index=True)
                #print(msg)
                #break
        prgBar.value = prgBar.value + 1
    df = df[~df.duplicated(subset=['text'], keep=False)]
    df.to_csv('data/'+group+'.csv')
    return df

In [4]:
def restore_data_from_file():
    return pd.read_csv('data.csv')

In [17]:
df = load_from_telegram('source_data/telegram.json','-4155924')
#df = restore_data_from_file()
len(df)

IntProgress(value=0, max=4)

40876

In [14]:
df.head

<bound method NDFrame.head of                     from     from_id  \
0      Vladimir Korotkov  4420223003   
1      Vladimir Korotkov  4420223003   
2          Алексей Богин  4348007839   
3          Алексей Богин  4348007839   
4          Алексей Богин  4348007839   
...                  ...         ...   
14933      Алексей Богин  4348007839   
14934    Александр Липин  4432196399   
14935            Николай  5420503156   
14936            Николай  5420503156   
14937            Николай  5420503156   

                                                    text  
0      взять этих долбоёбов Кокорина и Мамаева, каког...  
1      Наших тупорылых моряков за неделю закрыли в ко...  
2      Блет не успел написать чтоб его не приводил в ...  
3      В чем несостоятельность-то? Виновные понесли н...  
4      Если считаешь что недостаточно строгое обратис...  
...                                                  ...  
14933  С.Слепаков, «Америка нас не любит».\nПесня, ко...  
14934            

### prepare model

In [6]:
from deeppavlov import build_model, configs

In [7]:
model = build_model(configs.squad.squad_ru_rubert_infer, download=False)

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/alex/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/alex/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
/home/alex/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alex/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type

2020-11-02 11:54:31.162 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/alex/.deeppavlov/models/squad_ru_bert/model_rubert]
I1102 11:54:31.162680 140009968957248 tf_model.py:51] [loading model from /home/alex/.deeppavlov/models/squad_ru_bert/model_rubert]
W1102 11:54:31.164482 140009968957248 deprecation_wrapper.py:119] From /home/alex/.local/lib/python3.6/site-packages/deeppavlov/core/models/tf_model.py:54: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.



In [8]:
def get_answer(df,question):
    
    # select phrases
    pre_words = question.replace("?","").split(" ")
    words =[]
    for word in pre_words:
        if len(word)>2:
            words.append(word)    
    if len(words)==0:
        return 'Слишком короткий запрос'    
    
    df = df[df.text.str.contains('|'.join(words), regex = True)]
    
    if len(words)==0:
        return 'Нет данных'
    
    data = df.text.values.tolist()
    
    prgBar = IntProgress(min = 0, max = len(data))
    display(prgBar)
    
    answers = pd.DataFrame(columns=['answer','score'])
    for theme in data:
        res = model([theme], [question])
        answers = answers.append({'answer':res[0][0], 'score':res[2][0]}, ignore_index=True)            
        prgBar.value = prgBar.value + 1
        
    max_score = answers.score.max()    
    return answers[answers.score==max_score].iloc[0].answer

In [9]:
answer = get_answer(df,'Нужно ли сортировать мусор?')
answer

IntProgress(value=0, max=33)

'придётся'